In [1]:
!pip install fastembed

In [2]:
from fastembed import TextEmbedding

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
query =  'I just discovered the course. Can I join now?'

In [5]:
model_handle = 'jinaai/jina-embeddings-v2-small-en'

In [11]:
from fastembed import TextEmbedding
from qdrant_client import QdrantClient, models
embedding_model = TextEmbedding(model_name=model_handle)
embeddings_list = list(embedding_model.embed([query]))
min(embeddings_list[0])

np.float64(-0.11726373885183883)

In [12]:
embeddings_list1 = list(embedding_model.embed(['Can I still join the course after the start date?']))

In [13]:
embeddings_list[0].dot(embeddings_list1[0])

np.float64(0.9008528895674548)

In [14]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [16]:
docs_to_embed = []
for doc in documents:
    docs_to_embed.append(doc['text'])

embeddings_text = list(embedding_model.embed(docs_to_embed))
cosine = -1
for i,embedding in enumerate(embeddings_text):
    cosine = embedding.dot(embeddings_list[0])
    print(i, cosine)
    

0 0.762968451872193
1 0.8182378156620136
2 0.8085397445747489
3 0.7133078832064158
4 0.7304499196411823


In [17]:
text_to_embed = []
for doc in documents:
    text_to_embed.append(doc['question'] + ' ' + doc['text'])

embeddings_full_text = list(embedding_model.embed(text_to_embed))
for i,embedding in enumerate(embeddings_full_text):
    cosine = embedding.dot(embeddings_list[0])
    print(i, cosine)


0 0.851454319443226
1 0.8436594005975433
2 0.8408287224005012
3 0.7755157657626952
4 0.8086007917931166


In [19]:
import json
EMBEDDING_DIMENSIONALITY = 384

for model in TextEmbedding.list_supported_models():
    if model["dim"] == EMBEDDING_DIMENSIONALITY:
        print(json.dumps(model, indent=2))

{
  "model": "BAAI/bge-small-en",
  "sources": {
    "hf": "Qdrant/bge-small-en",
    "url": "https://storage.googleapis.com/qdrant-fastembed/BAAI-bge-small-en.tar.gz",
    "_deprecated_tar_struct": true
  },
  "model_file": "model_optimized.onnx",
  "description": "Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: necessary, 2023 year.",
  "license": "mit",
  "size_in_GB": 0.13,
  "additional_files": [],
  "dim": 384,
  "tasks": {}
}
{
  "model": "BAAI/bge-small-en-v1.5",
  "sources": {
    "hf": "qdrant/bge-small-en-v1.5-onnx-q",
    "url": null,
    "_deprecated_tar_struct": false
  },
  "model_file": "model_optimized.onnx",
  "description": "Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: not so necessary, 2023 year.",
  "license": "mit",
  "size_in_GB": 0.067,
  "additional_files": [],
  "dim": 384,
  "tasks": {}
}
{
  "model": "snowflake/snowflake-arctic-embed-xs",
  "sou

In [20]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [21]:
model =  "BAAI/bge-small-en"
from qdrant_client import QdrantClient, models
qd_client = QdrantClient("http://localhost:6333")
# Define the collection name
collection_name = "Hw2"
# Create the collection with specified vector parameters
qd_client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,  # Dimensionality of the vectors
        distance=models.Distance.COSINE  # Distance metric for similarity search
    )
)


True

In [22]:
qd_client.get_collections()

CollectionsResponse(collections=[CollectionDescription(name='zoomcamp-sparse-and-dense'), CollectionDescription(name='Hw2'), CollectionDescription(name='zoomcamp-rag'), CollectionDescription(name='zoomcamp-faq'), CollectionDescription(name='zoomcamp-sparse')])

In [23]:
qd_client.delete_collection(collection_name="zoomcamp-faq")

True

In [24]:
qd_client.delete_collection(collection_name="zoomcamp-sparse")

True

In [25]:
points = []
for i, doc in enumerate(documents):
    text = doc['question'] + ' ' + doc['text']
    vector=models.Document(text=text, model=model)
    point = models.PointStruct(
        id=i,
        vector = vector,
        payload=doc
    )
    points.append(point)

In [26]:
qd_client.upsert(
    collection_name=collection_name,
    points=points
)

Fetching 5 files: 100%|█████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 10.04it/s]


UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [32]:

query_points= qd_client.query_points(
        collection_name=collection_name,
        query=models.Document( 
            text=query,
            model=model 
        ),
        limit=5, 
        with_payload=True #to get metadata in the results
    )
    results = []
    
    for point in query_points.points:
        results.append(point.payload)
    return results

In [34]:
query_points.points[0]

ScoredPoint(id=14, version=0, score=0.87031734, payload={'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.', 'section': 'General course-related questions', 'question': 'The course has already started. Can I still join it?', 'course': 'machine-learning-zoomcamp'}, vector=None, shard_key=None, order_value=None)